In [5]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from torchvision import datasets
import cv2

# Memuat dataset Fashion MNIST
(train_images_mnist, train_labels_mnist), (test_images_mnist, test_labels_mnist) = fashion_mnist.load_data()
train_dataset = datasets.FashionMNIST('MNIST', train=True, download=True)
test_dataset = datasets.FashionMNIST('MNIST', train=False, download=True)
train_label_fashionmnist = train_dataset.targets
test_label_fashionmnist = test_dataset.targets

# Mengambil Subset label dari MNIST
train_label_fashionmnist = train_label_fashionmnist[:35552]
test_label_fashionmnist = test_label_fashionmnist[:8889]

# Path ke folder dataset
dataset_folder = 'images/'

# Path ke file styles.csv
styles_file = 'styles.csv'

# Membaca data dari file styles.csv
data = pd.read_csv(styles_file)

# Membaca gambar dan label dari folder dataset
images = []

# Memuat label ke dalam variabel
label_csv = data['articleType']

# Mengonversi label FashionImage menjadi tipe data string
label_csv = label_csv.astype(str)

# Mengambil Subset Label dari FashionImage
label_csv_train = label_csv[:35552]
label_csv_test = label_csv[35552:]

# Menggabungkan label FashionImages dengan label MNIST
train_label_gabungan = np.concatenate([train_label_fashionmnist, label_csv_train], axis=0)
test_label_gabungan = np.concatenate([test_label_fashionmnist, label_csv_test], axis=0)

# Menggabungkan gambar dari folder 'images' dengan dataset MNIST
image_folder = 'images/'

for filename in os.listdir(image_folder):
    image_path = os.path.join(image_folder, filename)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Mengubah gambar menjadi skala abu-abu (grayscale)
    image = cv2.resize(image, (28, 28))  # Menyesuaikan dimensi gambar menjadi 28x28
    image = np.expand_dims(image, axis=-1)  # Menambahkan dimensi kanal (1) pada gambar
    images.append(image)

# Konversi data gambar menjadi array numpy
images_data = np.array(images)

# Memastikan jumlah gambar dan label sama
train_images_mnist = np.expand_dims(train_images_mnist[:35552], axis=-1)
test_images_mnist = np.expand_dims(test_images_mnist[:8889], axis=-1)

# Mengambil Subset gambar dari FashionImage
images_data_train = images_data[:35552]
images_data_test = images_data[35552:]

# Menggabungkan data gambar dari FashionImage dan MNIST
x_train = np.concatenate([images_data_train, train_images_mnist], axis=0)
x_test = np.concatenate([images_data_test, test_images_mnist], axis=0)

# Mengubah label menjadi angka menggunakan LabelEncoder
label_encoder = LabelEncoder()
train_label_gabungan = label_encoder.fit_transform(train_label_gabungan.astype(str))
test_label_gabungan = label_encoder.transform(test_label_gabungan.astype(str))
num_classes = len(label_encoder.classes_)

# Membagi dataset menjadi data training, data test, dan data validasi
x_train, y_train = x_train, train_label_gabungan
x_test, x_val, y_test, y_val = train_test_split(x_test, test_label_gabungan, test_size=0.5, random_state=42)

# Normalisasi pixel gambar menjadi rentang [0, 1]
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
x_val = x_val.astype('float32') / 255

# Remove 'Face Moisturisers' label from training and test sets
indices_train = np.where(y_train != label_encoder.transform(['Face Moisturisers'])[0])
indices_test = np.where(y_test != label_encoder.transform(['Face Moisturisers'])[0])
x_train = x_train[indices_train]
y_train = y_train[indices_train]
x_test = x_test[indices_test]
y_test = y_test[indices_test]

# Label encoding
label_binarizer = LabelBinarizer()
y_train = label_binarizer.fit_transform(y_train)
y_test = label_binarizer.transform(y_test)
y_val = label_binarizer.transform(y_val)

# Membangun arsitektur model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Kompilasi model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Pelatihan model
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_val, y_val))

# Evaluasi model pada data test
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)



c:\Users\HP\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:511: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  np.object,


AttributeError: module 'numpy' has no attribute 'object'.
`np.object` was a deprecated alias for the builtin `object`. To avoid this error in existing code, use `object` by itself. Doing this will not modify any behavior and is safe. 
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
# Simpan model
model.save('image_classification_model.h5')
